In [1]:
# obtain raw assets
import os
os.system("dvc pull")

256

In [2]:
# initialize asset runner

import pandas as pd
from asset_runner import AssetRunner

runner = AssetRunner('../data/raw')

In [3]:
# process assets 
runner.process_assets()

--- Processing 3 assets ---
Name         File
-----------  ----------------------------
clubs        ../data/raw/clubs.json
appearances  ../data/raw/appearances.json
leagues      ../data/raw/leagues.json

-> Processing clubs
       club_id           name domestic_competition league_id
count      241            241                  241       241
unique     241            241                   14        14
top       1096  cercle-brugge                  TR1       TR1
freq         1              1                   21        21
All 1 validations passed!

-> Processing appearances
              goals       assists  yellow_cards     red_cards  minutes_played  \
count  73084.000000  73084.000000  73084.000000  73084.000000    73084.000000   
mean       0.090676      0.069755      0.142151      0.003982       67.203574   
std        0.319666      0.272337      0.360012      0.062976       30.557415   
min        0.000000      0.000000      0.000000      0.000000        0.000000   
25%        0

In [4]:
# list available assets
print(runner.prettify_asset_processors())

Name         File
-----------  ----------------------------
clubs        ../data/raw/clubs.json
appearances  ../data/raw/appearances.json
leagues      ../data/raw/leagues.json


In [5]:
# get a reference to one asset
asset = runner.assets[0]
asset['name']

'clubs'

In [6]:
# get a reference to the pandas dataframes containing raw and prepared data
clubs = asset['processor']

raw = clubs.raw_df
prep = clubs.prep_df

prep.describe()

,club_id,name,domestic_competition,league_id
count,241,241,241,241
unique,241,241,14,14
top,1096,cercle-brugge,TR1,TR1
freq,1,1,21,21


In [7]:
normalized = clubs.checkpoints['json_normalized']
normalized['parent.href'].str.split('/', 5, True)[4]

0       L1
1       L1
2       L1
3       L1
4       L1
      ... 
236    IT1
237    IT1
238    IT1
239    IT1
240    IT1
Name: 4, Length: 241, dtype: object

In [12]:
league = runner.assets[2]['processor']

json_normalized = league.get_checkpoint('json_normalized')
asset = pd.DataFrame()
league_href_parts = json_normalized['href'].str.split('/', 5, True)
asset['league_id'] = league_href_parts[4]
asset['name'] = 

,type,href,parent.type,parent.href
0,league,/bundesliga/startseite/wettbewerb/L1,confederation,/wettbewerbe/europa
1,league,/eredivisie/startseite/wettbewerb/NL1,confederation,/wettbewerbe/europa
2,league,/jupiler-pro-league/startseite/wettbewerb/BE1,confederation,/wettbewerbe/europa
3,league,/laliga/startseite/wettbewerb/ES1,confederation,/wettbewerbe/europa
4,league,/liga-nos/startseite/wettbewerb/PO1,confederation,/wettbewerbe/europa
5,league,/ligue-1/startseite/wettbewerb/FR1,confederation,/wettbewerbe/europa
6,league,/premier-league/startseite/wettbewerb/GB1,confederation,/wettbewerbe/europa
7,league,/premier-liga/startseite/wettbewerb/RU1,confederation,/wettbewerbe/europa
8,league,/premier-liga/startseite/wettbewerb/UKR1,confederation,/wettbewerbe/europa
9,league,/scottish-premiership/startseite/wettbewerb/SC1,confederation,/wettbewerbe/europa
